In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.optimize import minimize_scalar, fminbound
from scipy.misc import derivative

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Define symbolic variable
x = sp.Symbol('x')

## 1. Introduction to Derivatives

The **derivative** measures the rate of change of a function.

### Definition
$$f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

### Interpretations
1. **Instantaneous rate of change**
2. **Slope of tangent line** at point $(x, f(x))$
3. **Velocity** (if $f$ represents position)

### Notation
- $f'(x)$ (Lagrange)
- $\frac{df}{dx}$ (Leibniz)
- $Df(x)$ (Euler)
- $\dot{f}$ (Newton)

In [ ]:
def numerical_derivative(f, x, h=1e-5):
    """Compute derivative using limit definition"""
    return (f(x + h) - f(x)) / h

# Example: f(x) = x²
f = lambda x: x**2

# Calculate derivative at x = 2
x_point = 2
derivative_at_2 = numerical_derivative(f, x_point)

print(f"f(x) = x²")
print(f"f'(2) ≈ {derivative_at_2:.6f}")
print(f"Analytical: f'(x) = 2x, so f'(2) = 4")

# Visualize tangent line
x_vals = np.linspace(-1, 5, 200)
y_vals = f(x_vals)

# Tangent line: y - f(2) = f'(2)(x - 2)
tangent_slope = derivative_at_2
tangent_y = f(x_point) + tangent_slope * (x_vals - x_point)

plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_vals, 'b-', linewidth=2, label='f(x) = x²')
plt.plot(x_vals, tangent_y, 'r--', linewidth=2, label=f'Tangent at x=2 (slope={tangent_slope:.2f})')
plt.plot(x_point, f(x_point), 'ro', markersize=10, label=f'Point (2, 4)')
plt.grid(True, alpha=0.3)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Derivative as Slope of Tangent Line')
plt.legend()
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.show()

## 2. Derivative Rules

### Power Rule
$$\frac{d}{dx}(x^n) = nx^{n-1}$$

### Constant Multiple Rule
$$\frac{d}{dx}(cf(x)) = c\frac{d}{dx}(f(x))$$

### Sum/Difference Rule
$$\frac{d}{dx}(f(x) \pm g(x)) = f'(x) \pm g'(x)$$

### Product Rule
$$\frac{d}{dx}(f(x)g(x)) = f'(x)g(x) + f(x)g'(x)$$

### Quotient Rule
$$\frac{d}{dx}\left(\frac{f(x)}{g(x)}\right) = \frac{f'(x)g(x) - f(x)g'(x)}{(g(x))^2}$$

### Chain Rule
$$\frac{d}{dx}(f(g(x))) = f'(g(x)) \cdot g'(x)$$

In [ ]:
# Using SymPy for symbolic differentiation
x = sp.Symbol('x')

# Examples of different rules
functions = [
    (x**3, "Power Rule"),
    (3*x**2 + 2*x + 1, "Sum Rule"),
    (x**2 * sp.sin(x), "Product Rule"),
    (x**2 / (x + 1), "Quotient Rule"),
    ((x**2 + 1)**3, "Chain Rule"),
]

print("DERIVATIVE RULES DEMONSTRATIONS")
print("="*70)

for func, rule in functions:
    derivative = sp.diff(func, x)
    print(f"\n{rule}:")
    print(f"f(x)  = {func}")
    print(f"f'(x) = {derivative}")
    print(f"Simplified: {sp.simplify(derivative)}")

## 3. Critical Points

A **critical point** occurs where:
1. $f'(x) = 0$ (horizontal tangent), or
2. $f'(x)$ does not exist (sharp corner, vertical tangent)

### Types of Critical Points
- **Local Maximum**: Peak
- **Local Minimum**: Valley
- **Saddle Point**: Neither max nor min

### Finding Critical Points
1. Find $f'(x)$
2. Solve $f'(x) = 0$
3. Check where $f'(x)$ is undefined

In [ ]:
def find_critical_points(func_expr, symbol):
    """Find critical points of a function"""
    # Calculate derivative
    derivative = sp.diff(func_expr, symbol)
    
    # Solve f'(x) = 0
    critical_pts = sp.solve(derivative, symbol)
    
    return derivative, critical_pts

# Example: f(x) = x³ - 3x² - 9x + 5
x = sp.Symbol('x')
f_expr = x**3 - 3*x**2 - 9*x + 5

f_prime, critical_points = find_critical_points(f_expr, x)

print(f"Function: f(x) = {f_expr}")
print(f"Derivative: f'(x) = {f_prime}")
print(f"\nCritical points (f'(x) = 0):")
for cp in critical_points:
    f_value = f_expr.subs(x, cp)
    print(f"  x = {cp} = {float(cp):.4f}, f({float(cp):.4f}) = {float(f_value):.4f}")

## 4. First Derivative Test

Use the sign of $f'(x)$ to classify critical points:

### Test Procedure
For critical point $c$:
1. Check $f'(x)$ for $x < c$ (left side)
2. Check $f'(x)$ for $x > c$ (right side)

### Classification
- **Local Maximum**: $f'$ changes from $(+)$ to $(-)$
- **Local Minimum**: $f'$ changes from $(-)$ to $(+)$
- **Neither**: $f'$ does not change sign

### Function Behavior
- $f'(x) > 0$: Function is **increasing**
- $f'(x) < 0$: Function is **decreasing**
- $f'(x) = 0$: Function has **horizontal tangent**

In [ ]:
def first_derivative_test(func_expr, critical_point, symbol, delta=0.1):
    """Apply first derivative test to classify critical point"""
    derivative = sp.diff(func_expr, symbol)
    
    # Evaluate derivative slightly left and right of critical point
    left = derivative.subs(symbol, critical_point - delta)
    right = derivative.subs(symbol, critical_point + delta)
    
    left_val = float(left)
    right_val = float(right)
    
    print(f"\nFirst Derivative Test at x = {critical_point}:")
    print(f"  f'({float(critical_point) - delta:.2f}) = {left_val:.4f} ({'positive' if left_val > 0 else 'negative'})")
    print(f"  f'({float(critical_point) + delta:.2f}) = {right_val:.4f} ({'positive' if right_val > 0 else 'negative'})")
    
    if left_val > 0 and right_val < 0:
        return "Local Maximum"
    elif left_val < 0 and right_val > 0:
        return "Local Minimum"
    else:
        return "Neither (Saddle Point or Inflection)"

# Apply to our example
f_expr = x**3 - 3*x**2 - 9*x + 5
_, critical_points = find_critical_points(f_expr, x)

print("FIRST DERIVATIVE TEST")
print("="*60)

for cp in critical_points:
    classification = first_derivative_test(f_expr, cp, x)
    print(f"  Classification: {classification}")

In [ ]:
# Visualize the function with critical points
f_expr = x**3 - 3*x**2 - 9*x + 5
f_lambda = sp.lambdify(x, f_expr, 'numpy')
f_prime_expr = sp.diff(f_expr, x)
f_prime_lambda = sp.lambdify(x, f_prime_expr, 'numpy')

x_vals = np.linspace(-3, 5, 300)
y_vals = f_lambda(x_vals)
y_prime_vals = f_prime_lambda(x_vals)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot function
ax1.plot(x_vals, y_vals, 'b-', linewidth=2, label='f(x)')

# Mark critical points
_, critical_points = find_critical_points(f_expr, x)
for cp in critical_points:
    cp_val = float(cp)
    f_val = float(f_expr.subs(x, cp))
    classification = first_derivative_test(f_expr, cp, x)
    
    color = 'ro' if 'Maximum' in classification else 'go'
    ax1.plot(cp_val, f_val, color, markersize=12)
    ax1.annotate(f'{classification}\n({cp_val:.2f}, {f_val:.2f})', 
                xy=(cp_val, f_val), xytext=(20, 20),
                textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ax1.axhline(y=0, color='k', linewidth=0.5)
ax1.axvline(x=0, color='k', linewidth=0.5)
ax1.grid(True, alpha=0.3)
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')
ax1.set_title('Function f(x) = x³ - 3x² - 9x + 5')
ax1.legend()

# Plot derivative
ax2.plot(x_vals, y_prime_vals, 'r-', linewidth=2, label="f'(x)")
ax2.axhline(y=0, color='k', linewidth=2, linestyle='--', alpha=0.5, label='y = 0')

# Mark where derivative is zero
for cp in critical_points:
    cp_val = float(cp)
    ax2.plot(cp_val, 0, 'ro', markersize=10)
    ax2.axvline(x=cp_val, color='gray', linestyle=':', alpha=0.5)

# Shade regions
ax2.fill_between(x_vals, 0, y_prime_vals, where=(y_prime_vals > 0), 
                 alpha=0.3, color='green', label='f increasing (f\' > 0)')
ax2.fill_between(x_vals, 0, y_prime_vals, where=(y_prime_vals < 0), 
                 alpha=0.3, color='red', label='f decreasing (f\' < 0)')

ax2.grid(True, alpha=0.3)
ax2.set_xlabel('x')
ax2.set_ylabel("f'(x)")
ax2.set_title('Derivative f\'(x) = 3x² - 6x - 9')
ax2.legend()

plt.tight_layout()
plt.show()

## 5. Optimization Problems

Using derivatives to find maximum or minimum values.

### Steps for Optimization
1. **Identify** the quantity to optimize
2. **Express** it as a function of one variable
3. **Find** critical points using $f'(x) = 0$
4. **Classify** critical points (max, min, or neither)
5. **Check** endpoints if domain is restricted
6. **Verify** answer makes sense in context

In [ ]:
# Example: Maximum area problem
# A farmer has 100m of fence to enclose a rectangular area.
# Find dimensions for maximum area.

print("OPTIMIZATION PROBLEM: Maximize Rectangular Area")
print("="*60)
print("Given: 100m of fencing")
print("Find: Dimensions for maximum area\n")

# Let x = width, then length = (100 - 2x)/2 = 50 - x
# Area A(x) = x(50 - x) = 50x - x²

x = sp.Symbol('x', positive=True)
A = 50*x - x**2

print(f"Width: x")
print(f"Length: 50 - x")
print(f"Area function: A(x) = {A}")

# Find maximum
A_prime = sp.diff(A, x)
critical_points = sp.solve(A_prime, x)

print(f"\nDerivative: A'(x) = {A_prime}")
print(f"Critical point: x = {critical_points[0]}")

optimal_width = float(critical_points[0])
optimal_length = 50 - optimal_width
max_area = float(A.subs(x, optimal_width))

print(f"\nOptimal dimensions:")
print(f"  Width: {optimal_width} m")
print(f"  Length: {optimal_length} m")
print(f"  Maximum area: {max_area} m²")
print(f"  Shape: Square!")

# Visualize
x_vals = np.linspace(0, 50, 200)
A_vals = 50*x_vals - x_vals**2

plt.figure(figsize=(10, 6))
plt.plot(x_vals, A_vals, 'b-', linewidth=2, label='Area A(x) = 50x - x²')
plt.plot(optimal_width, max_area, 'ro', markersize=12, 
         label=f'Maximum: ({optimal_width}, {max_area})')
plt.axvline(x=optimal_width, color='r', linestyle='--', alpha=0.5)
plt.axhline(y=max_area, color='r', linestyle='--', alpha=0.5)
plt.grid(True, alpha=0.3)
plt.xlabel('Width (m)')
plt.ylabel('Area (m²)')
plt.title('Area vs Width: Finding Maximum')
plt.legend()
plt.xlim(0, 50)
plt.ylim(0, max_area + 100)
plt.show()

## 6. Practice Problems

In [ ]:
# Problem 1: Find and classify critical points
print("Problem 1: f(x) = x⁴ - 4x³")
print("="*60)

x = sp.Symbol('x')
f1 = x**4 - 4*x**3
f1_prime = sp.diff(f1, x)
crit_pts = sp.solve(f1_prime, x)

print(f"f(x) = {f1}")
print(f"f'(x) = {f1_prime}")
print(f"Critical points: {crit_pts}")

for cp in crit_pts:
    if cp != 0:  # Skip x=0 for this example
        classification = first_derivative_test(f1, cp, x)
        f_val = float(f1.subs(x, cp))
        print(f"\nx = {cp}: f({cp}) = {f_val:.2f}")
        print(f"Classification: {classification}")

# Problem 2: Optimization
print("\n\nProblem 2: Minimize surface area of cylindrical can")
print("="*60)
print("Volume = 1000 cm³")
print("Find: radius and height to minimize surface area")
print("\nSolution:")
print("V = πr²h = 1000")
print("h = 1000/(πr²)")
print("Surface Area: S = 2πr² + 2πrh")
print("Substituting: S(r) = 2πr² + 2000/r")

r = sp.Symbol('r', positive=True)
S = 2*sp.pi*r**2 + 2000/r
S_prime = sp.diff(S, r)
optimal_r = sp.solve(S_prime, r)

# Filter for positive real solution
optimal_r_val = [float(sol.evalf()) for sol in optimal_r if sol.is_real and sol > 0][0]
optimal_h_val = 1000 / (np.pi * optimal_r_val**2)

print(f"\nOptimal radius: {optimal_r_val:.2f} cm")
print(f"Optimal height: {optimal_h_val:.2f} cm")
print(f"Notice: height = 2 × radius")

## Summary

### Key Concepts
1. **Derivative**: Rate of change, slope of tangent line
2. **Derivative Rules**: Power, product, quotient, chain rules
3. **Critical Points**: Where $f'(x) = 0$ or undefined
4. **First Derivative Test**: Classify critical points using sign changes
5. **Optimization**: Using derivatives to find max/min values

### Important Formulas
- Definition: $f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$
- Power Rule: $\frac{d}{dx}(x^n) = nx^{n-1}$
- Product Rule: $(fg)' = f'g + fg'$
- Chain Rule: $(f(g(x)))' = f'(g(x)) \cdot g'(x)$

### Problem-Solving Strategy
1. Find $f'(x)$
2. Solve $f'(x) = 0$ for critical points
3. Use first derivative test to classify
4. Check endpoints if applicable

### Next Week
Week 09: Integrals & Areas